In [1]:
import gradio as gr
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('vet_guardian2.csv')

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

 
def get_most_similar_disease(user_symptoms, user_species):
    disease=[]
    diseaseSim=[]
    user_species = user_species.lower()
    filtered_df = df[df['Species'] == user_species]
    label=None
    for label in filtered_df['Symptoms'].unique():
        sentence1 = user_symptoms
        sentence2 = label

        # Convert the sentences into vector embeddings
        embedding1 = model.encode(sentence1, convert_to_tensor=True)
        embedding2 = model.encode(sentence2, convert_to_tensor=True)

        # Calculate the cosine similarity between the two embeddings
        cos_sim = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
        cos_sim = cos_sim*100
        #print(label," and ",cos_sim)
        disease.append(label)
        diseaseSim.append(cos_sim)
    return disease,diseaseSim,filtered_df

def predict_disease(symptoms, species):
    try:
        disease, diseaseSim, filtered_df = get_most_similar_disease(symptoms, species)
        sorted_indexes = np.argsort(diseaseSim)[::-1]  # Sort indexes based on similarity in descending order
        result = ""
        for index in sorted_indexes[:3]:  # Select the top 3 indexes
            symptom = disease[index]
            other_symptoms = symptom
            other_symptoms = ', '.join(symptom for symptom in other_symptoms.split('|') if symptom != symptoms)
            disease_name = filtered_df[filtered_df['Symptoms'] == symptom]['Disease_Name'].values[0]
            disease_description = filtered_df[filtered_df['Symptoms'] == symptom]['Disease_Description'].values[0]
            prevention_measures = filtered_df[filtered_df['Symptoms'] == symptom]['Prevention_Measures'].values[0]
            result += f"If your pet is also experiencing symptoms like {other_symptoms},\n"
            result += f"Disease: {disease_name}\n\n"
            result += f"Description: {disease_description}\n\n"
            result += f"Prevention Measures: {prevention_measures}\n\n"
            result += "----------------------------------------\n\n"
        return result
    except IndexError:
        print(f'Error: Disease not found for symptoms: {symptoms} in species: {species}. Please consult a veterinarian.')
        return "Error: Disease not found. Please consult a veterinarian."
           
# Define species options
species_options = ["Cat", "Dog", "Hamster", "Guinea Pigs", "Cow", "Ferret", "Turtle", "Rabbit", "parrot", "Hen" ]

# Define the Gradio interface with dropdown for species selection
interface = gr.Interface(
    fn=predict_disease,
    allow_flagging="never",
    inputs=[gr.Textbox(label="Symptoms"), gr.Dropdown(label="Species", choices=species_options)],
    outputs="text",
    title="VET GUARDIAN",
    description="Enter the symptoms and select the species to get the disease. Format the input as 'symptoms species' (e.g., 'coughing cat')"
)
# Launch the Gradio interface
interface.launch()

C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\DELL\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.21.0, however version 4.29.0 is available, please upgrade.
--------
